## Data Exploration

In [ ]:
import pandas as pd

In [ ]:
"""Reading data"""

df_outcome = pd.read_csv('http://www.sharecsv.com/dl/e593dd089bc5aaf6583b5187f96cb48e/outcome_interview.csv')
df_event   = pd.read_csv('http://www.sharecsv.com/dl/f9b678bf4a60bafe3f0a4132313e194d/event_interview.csv')
df_fighter = pd.read_csv('http://www.sharecsv.com/dl/4f1b611a0d9f96d7700adba8b387df21/fighter_interview.csv')

In [ ]:
"""Data Exploration: check the what variables dataframe has"""
df_outcome.head()


In [ ]:
df_event.head()

In [ ]:
df_fighter.head()

In [ ]:
"""
    The objective is to get all matchups containing fighters' names, nationalities and an 
    event name they participated.
    
    The outcome table must have the following structure:    
    | fighter1_name | fighter2_name | fighter1_nationality | fighter2_nationality | event_name |
    
    Therefore, we will join events with outcomes on the event id, and then join the resulted table twice with 'fighter' 
    table on the fighter1_id and fighter2_id

""" 
df_merged = \
        df_event.merge(df_outcome, 
               left_on='id', 
               right_on='event_id', 
               suffixes=['_event','_outcome'])\
        .merge(df_fighter,
              left_on = 'fighter1_id',
              right_on= 'id',
              suffixes=['_event_outcome', '_fighter1'])\
        .merge(df_fighter,
              left_on = 'fighter2_id',
              right_on= 'id',
              suffixes=['_event_outcome_fighter1', '_fighter2'])

        
"""Print all columns that are in the joined table"""
df_merged.columns

In [ ]:
"""Select columns that we are interested in and rename them"""
df_matchups = df_merged[['name_fighter1', 'name', 'nationality_event_outcome_fighter1', 'nationality_fighter2', 'name_event_outcome']]
df_matchups.columns = ['fighter1_name', 'fighter2_name', 'fighter1_nationality', 'fighter2_nationality', 'event_name']
df_matchups.head()


In [ ]:
"""
    First row looks suspicious. An event called Legacy FC 45: Todhunter vs. Smith have fighters 
    Ed Cline and JC Cottrell, but logically, it should be Todhunter and Smith. Simple googling 
    tells that the main fight Todhunter vs. Smith has been canceled due to Todhunter's knee injury and has been replaced
    with Cline vs. Cottrell. Further research shows that some events do not contain all fights that took place
    
    Nevertheless, let's sort data by event_name to see all fighters of the event:
"""
df_matchups = df_matchups.sort_values('event_name')
df_matchups.head(10)


In [ ]:
"""Now it looks better. Let's save results to csv file"""
df_matchups.to_csv('matchups.csv', index=False)

In [ ]:
"""
    The event names do not seem to be exactly correct. 
    Some event are named after the promoter along with the event edition and fighters' names.
    For example, event KSW 42: Narkun vs. Khalidov. Let's check its outcome:
    
"""
df_event[df_event.name=='KSW 42: Narkun vs. Khalidov']
df_outcome[df_outcome.event_id==1312]

In [ ]:
"""
    There are several fights happen in that event. Let's find out who are the fighters
"""
df_outcome[df_outcome.event_id==1312]\
    .merge(df_fighter, left_on='fighter1_id', right_on='id')\
    .merge(df_fighter, left_on='fighter2_id', right_on='id')

Hence, the event called 'KSW 42: Narkun vs. Khalidov' has 7 fights, including Narkun and Halidov, 
but there are other fighters as well. Therefore, it got me confused. I checked the event page:
http://www.kswmma.com/en/index.php?s=news&newsID=2551&ksw-42-results
and found that fights from the results above are in there


## Distribution of fighters across nationalities

In [ ]:
"""
    We will use the results to plot the distribution of fighters across countries. First, we stack the subsets of
    dataframes for each of fighter with his nationality
"""
df_part_fighters = pd.concat([df_matchups[['fighter1_name', 'fighter1_nationality']],
                              df_matchups[['fighter2_name', 'fighter2_nationality']]\
                              .rename(columns = {'fighter2_name': 'fighter1_name',
                                                 'fighter2_nationality': 'fighter1_nationality'
                                                }
                                     )
                             ],
                             axis=0, ignore_index=True
                   )
df_part_fighters = pd.concat([df_matchups[['fighter1_name', 'fighter1_nationality']],
                              df_matchups[['fighter2_name', 'fighter2_nationality']]\
                              .rename(columns = {'fighter2_name': 'fighter1_name',
                                                 'fighter2_nationality': 'fighter1_nationality'
                                                }
                                     )
                             ],
                             axis=0, ignore_index=True
                   )
df_part_fighters.columns = ['name', 'nationality']

In [ ]:
"""
    Before plotting, we need to do some data wrangling
    1. Check if there are rows with NAs and drop them in this case
    2. Some countries can have a variety of names. We have to check what countries are in the dataset
"""
df_part_fighters = df_part_fighters.dropna(axis=0)
df_part_fighters.nationality.unique()

In [ ]:
"""
    There are ambiguous names: USA/United States/United states, Holland/Netherlands - merge them 
"""
df_part_fighters.loc[(df_part_fighters.nationality=='United States')\
                     |(df_part_fighters.nationality=='United states'), 'nationality'] = 'United States of America'
df_part_fighters.loc[df_part_fighters.nationality=='Holland', 'nationality'] = 'Netherlands'

In [ ]:
df_country_data = df_part_fighters\
                  .drop_duplicates('name')\
                  .groupby(by='nationality', 
                           as_index=False)\
                  .count()\
                  .sort_values('name', ascending=False)
df_country_data.columns=['country', 'count']

#Print top 5 countries
df_country_data.head()

In [ ]:
"""
    How many different nationalities there are in the data
"""
df_country_data.country.unique().shape

In [ ]:
"""
    There are 112 countries in total. To plot categorical data across countries, we will use horizontal bar chart
    Let's plot top 25 countries, for the start
"""
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="whitegrid")

# Initialize the matplotlib figure
f, ax = plt.subplots(figsize=(10, 15))

sns.set_color_codes()
g = sns.barplot(x="count", y="country", data=df_country_data.iloc[:25],
            label="count", color="b", orient='h')\
       .set_title("Distribution of fighters' nationalities")

# Add a legend and informative axis label
ax.legend(loc="lower right", frameon=True)
ax.set(ylabel="Country",
       xlabel="Number of fighters")
sns.despine(left=True, bottom=False)

In [ ]:
"""
    Barchart suggests that top-10 countries has most of the fighters (USA, Brazil, England, Japan ...) 
    
    There also an option to build an interactive pie chart, as a way to visualise categorical data
"""

#pip install python-nvd3

from nvd3 import pieChart
output_file = open('distribution.html', 'w')
cutoff = 25

chart = pieChart(name='pieChart', color_category='category20c', height=450, width=450)
xdata = df_country_data['country'].tolist()[:cutoff]
ydata = df_country_data['count'].tolist()[:cutoff]
extra_serie = {"tooltip": {"y_start": "", "y_end": " cal"}}
chart.add_serie(y=ydata, x=xdata, extra=extra_serie)
chart.buildhtml()
output_file.write(chart.htmlcontent)
output_file.close()

"""
    For the best rendering open html file in the project folder
"""
chart